In [119]:
# Librerias y dependencias utilizadas
import numpy as np
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [120]:
df =  pd.read_csv("DataOpt/final_dataset_join.csv")
dff = df.copy()
df_class = dff.Grupo
df_grupos= dff.drop(["Grupo"], axis=1)
scaler.fit(df_grupos)
df_x = scaler.transform(df_grupos)



In [121]:
class definicion_pesos:

  @staticmethod
  def evaluate(cells):
    rows = df_x.shape[0]
    pred_class = []
    for i in range(rows):
      current_row = df_x[i]
      resta = cells * np.power(df_x - current_row, 2)
      suma_filas = resta.sum(axis=1)
      suma_filas = np.sqrt(suma_filas)

      posiciones = np.arange(0, suma_filas.shape[0])
      vector_unido = np.column_stack((posiciones, suma_filas))
      vector_unido = np.delete(vector_unido, i, axis=0) #borra la fila que se esta evaluando

      indices_ordenamiento = vector_unido[:, 1].argsort()
      vector_unido = vector_unido[indices_ordenamiento]
      pred_class.append(df_class[vector_unido[0][0]])

    acc = accuracy_score(df_class, pred_class)
    return acc

  def __str__(self):
    return "Sphere-lb:"


class solution:

    def __init__(self, d: int, f):
        self.size = d
        self.cells = np.zeros(self.size, float)
        self.fitness = 0.0
        self.function = f

    def from_solution(self, origin):
        self.size = origin.size
        self.cells = np.copy(origin.cells)
        self.fitness = origin.fitness
        self.function = origin.function

    def Initialization(self):
        # vector de d dimensiones incicalizado aleatoriamente con valores entre (0, 1) normalizado
        self.cells = np.random.uniform(low=0, high=1, size=(self.size,))
        sum = self.cells.sum()
        self.cells = self.cells / sum

        self.fitness = self.function.evaluate(self.cells)
        # print("Vector de d dimensiones inicializado : ", self.cells)
        # print("Fitness : ", self.fitness)

    def random_not_in_list(self, d: int, vector_a_excluir):
      valor = np.random.randint(0, d)
      while valor in vector_a_excluir:
          valor = np.random.randint(0, d)
      return valor

    def tweak(self, bandwidth: float, swaps:int):
        posiciones = np.random.choice(self.size, swaps, replace=False)
        for pos in posiciones:
          bw = np.random.uniform(low=-bandwidth, high=bandwidth, size=(1,))[0]
          new_val = self.cells[pos] + bw
          while new_val < 0 or new_val > 1:
            bw = np.random.uniform(low=-bandwidth, high=bandwidth, size=(1,))[0]
            new_val = self.cells[pos] + bw
          self.cells[pos] = new_val

          pos_to_swap = self.random_not_in_list(self.size, posiciones)
          new_val = self.cells[pos_to_swap] - bw
          while new_val < 0 or new_val > 1:
            pos_to_swap = self.random_not_in_list(self.size, posiciones)
            new_val = self.cells[pos_to_swap] - bw
          self.cells[pos_to_swap] = self.cells[pos] - bw

        self.fitness = self.function.evaluate(self.cells)

    def __str__(self):
        return "cells:" + str(self.cells) + \
               "-fit:" + str(self.fitness)


class SA:
    def __init__(self, max_efos: int, bandwidth: float, swaps: int):
        self.max_efos = max_efos
        self.bandwidth = bandwidth
        self.swaps = swaps

    def evolve(self, seed: int, d: int, f):
        to = 100
        self.function = f
        np.random.seed (seed)
        best_fitness_history = np.zeros(self.max_efos, float)

        S = solution(d, f) # S is a new Solution
        S.Initialization()
        best_fitness_history[0] = S.fitness
        self.best = solution(d, f)
        self.best.from_solution(S) # self.best is a full copy of S
        t= to
        for iteration in range(1, self.max_efos):
            R = solution(S.size, S.function)
            R.from_solution(S) # R is a full copy of S
            R.tweak(self.bandwidth, self.swaps)
            t = t - to/(self.max_efos + 1)
            ale = np.random.uniform()
            prob = np.exp((R.fitness - S.fitness) / t) # Maximizing
            if R.fitness > S.fitness or ale < prob: # Maximizing
                S.from_solution(R)
            if S.fitness > self.best.fitness: # Maximizing
                self.best.from_solution(S)
            best_fitness_history[iteration] = self.best.fitness
            if iteration % 100 == 0:
              print("EFO: " + str(iteration) + " - Fitness: " + str(self.best.fitness))
        return best_fitness_history
        # revisar si se estanca y hacer otro arranque desde ese punto .. multi star

    def __str__(self):
        result = "SA:-bandwidth:" + str(self.bandwidth)
        return result



In [123]:
# Plot convergence curve
def plot_convergence_curve(fitness_history, f, alg):
  efos = np.arange(len(fitness_history))
  plt.title("Convergence curve for " + str(f))
  plt.xlabel("EFOs")
  plt.ylabel("Fitness")
  plt.plot(efos, fitness_history, label=str(alg))
  plt.legend()
  plt.savefig("Convergence curve for " + str(f) + "" + str(alg) + ".png")
  plt.show()

In [28]:
att = df_x.shape[1]
print(att)
f = definicion_pesos()
np.random.seed(42)
w = np.random.uniform(low=0, high=1, size=(att,))
sum = w.sum()
w = w / sum
#print (w)
t = f.evaluate(w)
t

174


0.3662714097496706

In [37]:
# d numero de atributos
d = 174
f = definicion_pesos()
max_efos = 30000
bw = 1/d/20
seed = 42
swaps = 5
sa = SA(max_efos=max_efos, bandwidth=bw, swaps=swaps)
print(sa)
start_timer = time.time()
curve = sa.evolve(seed=seed, d=d, f=f)
end_timer = time.time()
print("Elapsed time: ", end_timer - start_timer)
best = sa.best
print(best.fitness)
plot_convergence_curve(curve, f, sa)

SA:-bandwidth:0.00028735632183908046
EFO: 100 - Fitness: 0.3820816864295125
EFO: 200 - Fitness: 0.4031620553359684
EFO: 300 - Fitness: 0.4031620553359684
EFO: 400 - Fitness: 0.4031620553359684
EFO: 500 - Fitness: 0.4031620553359684
EFO: 600 - Fitness: 0.4031620553359684


KeyboardInterrupt: 